In [2]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# updating nipype from 1.7.0 -> 1.8.1, will hopefully ifx no PE estimates founds error # didn't fix

220922-11:45:57,790 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

In [2]:
sstsubs = sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', 'sub-*', 'ses-sstmsit', 'func','fwhm-1p5','model-0','*task-sst_space-MNI*contrast-0_desc-zstat*')))
print(f'Group level based on {len(sstsubs)} subjects\n')
print([x.split('/')[-2].split('-')[-1] for x in sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'fmriprep','fmriprep', 'sub-*', 'ses-sstmsit')))])

sst = [x.split('/')[-6].split('-')[-1] for x in sstsubs]
allsst = [x.split('/')[-2].split('-')[-1] for x in sorted(glob.glob(os.path.join('/home/Public/trondheim', 'sourcedata', 'zipdata', 'sub-*', 'ses-sstmsit')))]
print(f'\nsubs {sorted(list(set(allsst) - set(sst)))} are missing')

# remove subs who did it wrong
# to_remove = ['004', '010', '008', '019', '013', '027']
print(f"\nsubs: {(to_remove := sorted(['004', '010', '008', '019', '013', '027']))} were excluded")

print(f'\ncomplete sub list is (n={len((fin_sub := [x for x in allsst if x not in to_remove]))}):\n\n {fin_sub}')

Group level based on 31 subjects

['002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '023', '024', '025', '026', '027', '029', '031', '032', '033', '034', '035', '037', '038', '039', '040', '041', '042']

subs ['004', '008', '010', '013', '019', '027'] are missing

subs: ['004', '008', '010', '013', '019', '027'] were excluded

complete sub list is (n=31):

 ['002', '003', '005', '006', '007', '009', '011', '012', '014', '015', '016', '017', '018', '020', '021', '023', '024', '025', '026', '029', '031', '032', '033', '034', '035', '037', '038', '039', '040', '041', '042']


In [3]:
# general set-up
project_folder = '/home/Public/trondheim'
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
# The directory where we can find the level 2 output
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model')


spaces = ['T1w']   # shouldn't touch this but just in case we _do_ want to go back to MNI....
ses = 'sstmsit'      # don't make this a list, that won't work
task = 'sst'     # don't make this a list, that won't work
model_ns = ['0']      # 
fwhms = ['1p5']#, '4p5']

template_brain = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
template_brain_mask = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii'

# subject_ids = sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat', 'subject_level_model', 'sub-*', f'ses-{ses}')))
# subject_ids = sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'glm_feat_hp', 'subject_level_model', 'sub-*', 'ses-sstmsit', 'func','fwhm-1p5',f'model-{model_ns}','*task-sst_space-MNI*contrast-0_desc-zstat*')))
# subject_ids = [x.split('/')[-6].split('-')[-1] for x in subject_ids]
# subject_ids = [x for x in subject_ids if x not in ['021','023','024','026','036']]
subject_ids = fin_sub

if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
#     contrasts = ['3','4','5']
#     contrasts = ['0']
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model


templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', f'ses-{ses}', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-sstmsit_task-sst_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', f'ses-{ses}', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-sstmsit_task-sst_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', f'ses-{ses}', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-sstmsit_task-sst_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}

## if uneven number of template / designs / subs the code will crash
print(f"no.copes = {len(glob.glob(templates['level2_cope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}")
print(f"no.varvopes = {len(glob.glob(templates['level2_varcope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}")
print(f"no.tdofs = {len(glob.glob(templates['level2_tdof'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}")

print(subject_ids)

no.copes = 31
no.varvopes = 31
no.tdofs = 31
['002', '003', '005', '006', '007', '009', '011', '012', '014', '015', '016', '017', '018', '020', '021', '023', '024', '025', '026', '029', '031', '032', '033', '034', '035', '037', '038', '039', '040', '041', '042']


In [4]:
len(subject_ids)

31

In [5]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [6]:
##
workflow = pe.Workflow(name='feat_level3_sst_hp')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [7]:
def get_subject_ids():
    import glob
    import os
    subject_ids = sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', 'sub-*', 'ses-sstmsit', 'func','fwhm-1p5','model-1','*task-sst_space-MNI*contrast-0_desc-zstat*')))
    subject_ids = [x.split('/')[-6].split('-')[-1] for x in subject_ids]
    to_remove = ['004', '010', '008', '019', '013', '027']
    subject_ids =  [x for x in subject_ids if x not in to_remove]
    print(f'number of subs: {len(subject_ids)}')
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids): #=subject_ids):
    print(second_level_contrast)
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors


contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')



In [8]:
# flameo = pe.Node(
#     interface=fsl.FLAMEO(run_mode='flame1'),
#     name="flame1")

# flameo.inputs.mask_file = template_brain_mask

# workflow.connect([
#     (copemerge, flameo, [('merged_file', 'cope_file')]),
#     (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
#     (level3model, flameo, [('design_mat', 'design_file'),
#                            ('design_con', 't_con_file'), 
#                            ('design_grp', 'cov_split_file')]),
# ])

flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [9]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [10]:
# ## Datasink
# datasink = pe.Node(nio.DataSink(), name='sinker')
# datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-sstmsit","task-sst")

# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

# ## todo: substitutions
# workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
# workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
# workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
# workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

# ## cluster results
# workflow.connect(grf_cluster, 'threshold_file', datasink, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'grf_localmax_txt_file')
# workflow.connect(grf_cluster, 'index_file', datasink, 'cluster_indices')

In [11]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "group_level_model", "ses-sstmsit","task-sst","model-1",'model-1')

substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                         'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                        (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                          'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                        # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                        #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                       ]

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 20, 'memory_gb': 200})

220922-11:46:00,237 nipype.workflow INFO:
	 Workflow feat_level3_sst_hp settings: ['check', 'execution', 'logging', 'monitoring']
220922-11:46:00,346 nipype.workflow INFO:
	 Running in parallel.
220922-11:46:00,357 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 200.00/200.00, Free processors: 20/20.
220922-11:46:00,441 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.subject_id_getter" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/subject_id_getter".
220922-11:46:00,442 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_0_fwhm_1p5_model_n_0/selector".
220922-11:46:00,443 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_1_fwhm_1p5_model_n_0/selector".
220922-11:46:00,443 n

220922-11:46:02,435 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.copemerge" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_1_fwhm_1p5_model_n_0/copemerge".
220922-11:46:02,436 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.copemerge" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_2_fwhm_1p5_model_n_0/copemerge".
220922-11:46:02,507 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l3" <nipype.interfaces.utility.wrappers.Function>
220922-11:46:02,507 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l3" <nipype.interfaces.utility.wrappers.Function>
220922-11:46:02,507 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l3" <nipype.interfaces.utility.wrappers.Function>
220922-11:46:02,508 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l3" <nipype.interfaces.utility.wrappers.Function>
220922-11:46:02,508 nipype.workflow INFO:
	 [Node] Executing "cont

220922-11:46:25,952 nipype.workflow INFO:
	 [Node] Finished "copemerge", elapsed time 23.440051s.
220922-11:46:26,383 nipype.workflow INFO:
	 [Job 24] Completed (feat_level3_sst_hp.copemerge).
220922-11:46:26,385 nipype.workflow INFO:
	 [MultiProc] Running 11 tasks, and 0 jobs ready. Free memory (GB): 197.80/200.00, Free processors: 9/20.
                     Currently running:
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
220922-

220922-11:51:58,715 nipype.workflow INFO:
	 [Job 36] Completed (feat_level3_sst_hp.flameo).
220922-11:51:58,717 nipype.workflow INFO:
	 [MultiProc] Running 11 tasks, and 1 jobs ready. Free memory (GB): 197.80/200.00, Free processors: 9/20.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
220922-11:51:59,853 nipype.workflow INFO:
	 [Node] Finished "flameo", elapsed time 329.316004s.
220922-11:52:00,717 nipype.workflow INFO:
	 [Job 32] Completed 

220922-11:52:06,786 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster0" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_2_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_2.3/grf_cluster/mapflow/_grf_cluster0".
220922-11:52:06,786 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_2_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_2.3/grf_cluster/mapflow/_grf_cluster1".
220922-11:52:06,787 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster0" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_2_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_3.1/grf_cluster/mapflow/_grf_cluster0".
220922-11:52:06,787 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_sst_hp/_contrast_n_2_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_

...

220922-11:52:18,857 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/third_level_model/grf_thresholded_zstats_file/_contrast_n_3_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_3.1/_grf_cluster1/zstat2_threshold.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/model-0/model-0_fwhm-1p5_subjectlevelcontrast-3_grouplevelcontrast-2_flame-1_desc-zstat_voxelthreshold-3.1.nii.gz
220922-11:52:18,872 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>
220922-11:52:18,872 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>
220922-11:52:18,894 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
220922-11:52:18,894 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
220922-11:52

220922-11:52:19,215 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.364373s.
220922-11:52:20,19 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 1.14569s.
220922-11:52:20,114 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 1.241393s.
220922-11:52:20,156 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 1.260184s.
220922-11:52:20,298 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 1.4025020000000001s.
220922-11:52:20,735 nipype.workflow INFO:
	 [Job 72] Completed (feat_level3_sst_hp.grf_cluster).
220922-11:52:20,736 nipype.workflow INFO:
	 [Job 73] Completed (feat_level3_sst_hp.grf_cluster).
220922-11:52:20,737 nipype.workflow INFO:
	 [Job 84] Completed (feat_level3_sst_hp.sinker).
220922-11:52:20,739 nipype.workflow INFO:
	 [Job 85] Completed (feat_level3_sst_hp.sinker).
220922-11:52:20,740 nipype.workflow INFO:
	 [Job 92] Completed (feat_level3_sst_hp.sinker).
220922-11:52:20,741 nipy

220922-11:52:20,878 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/third_level_model/level3_tdof_ts/_contrast_n_4_fwhm_1p5_model_n_0/_run_mode_flame1/tdof_t2.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/model-0/model-0_fwhm-1p5_subjectlevelcontrast-4_grouplevelcontrast-2_flame-1_desc-tdof_t.nii.gz
220922-11:52:20,879 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/third_level_model/grf_thresholded_zstats_file/_contrast_n_4_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_3.1/_grf_cluster0/zstat1_threshold.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/model-0/model-0_fwhm-1p5_subjectlevelcontrast-4_grouplevelcontrast-1_flame-1_desc-zstat_voxelthreshold-3.1.nii.gz
220922-11:52:

220922-11:52:22,873 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/third_level_model/level3_zstats/_contrast_n_5_fwhm_1p5_model_n_0/_run_mode_flame1/zstat2.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/model-0/model-0_fwhm-1p5_subjectlevelcontrast-5_grouplevelcontrast-2_flame-1_desc-zstat.nii.gz
220922-11:52:22,874 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/third_level_model/level3_copes/_contrast_n_5_fwhm_1p5_model_n_0/_run_mode_flame1/cope1.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-sst/model-1/model-1/model-0/model-0_fwhm-1p5_subjectlevelcontrast-5_grouplevelcontrast-1_flame-1_desc-cope.nii.gz
220922-11:52:22,875 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_fe

220922-11:52:24,740 nipype.workflow INFO:
	 [Job 101] Completed (feat_level3_sst_hp.sinker).
220922-11:52:24,742 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 0 jobs ready. Free memory (GB): 198.80/200.00, Free processors: 14/20.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo


In [14]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/crashdumps/crash-20220725-214615-scotti-flameo.bI.b1.c2-99721cf0-cd99-49b3-8229-f26afd04b843.pklz')
res

{'node': feat_level3_sst_hp.flameo.bI.b1.c2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node flameo.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in ru

ERROR! Session/line number was not unique in database. History logging moved to new session 261


In [15]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/crashdumps/crash-20220423-024121-scotti-flameo.bI.b0.c2-f70d4e1b-b715-4a19-8f4e-64dc66a64cc5.pklz')
res

{'node': feat_level3_sst.flameo.bI.b0.c2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/miniconda3/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node flameo.\n\nTraceback (most recent call last):\n  File "/home/scotti/miniconda3/envs/py38/lib/pytho

In [2]:
# # subject_ids = ['%02d' % i for i in np.arange(1, 20)]

# templates = {'level2_cope':'/home/gdholla1/projects/bias/data/derivatives/glm_fits_level2/level2_copes/_mask_{mask}_subject_id_{subject_id}/_fwhm_{fwhm}/_shift_{shift}/_flameo{contrast}/cope1.nii.gz',
#              'level2_varcope':'/home/gdholla1/projects/bias/data/derivatives/glm_fits_level2/level2_varcopes/_mask_{mask}_subject_id_{subject_id}/_fwhm_{fwhm}/_shift_{shift}/_flameo{contrast}/varcope1.nii.gz',
#              'level2_tdof':'/home/gdholla1/projects/bias/data/derivatives/glm_fits_level2/level2_tdof/_mask_{mask}_subject_id_{subject_id}/_fwhm_{fwhm}/_shift_{shift}/_flameo{contrast}/tdof_t1.nii.gz',             
#              'epi2struct':'/home/gdholla1/projects/bias/data/derivatives/registration/epi2t1weighted/epi2structmat_ants/_subject_id_{subject_id}/transformComposite.h5',
#              't1weighted':'/home/gdholla1/projects/bias/data/raw/sub-{subject_id}/anat/sub-{subject_id}_T1w.nii.gz',
#              'struct2mni':'/home/gdholla1/projects/bias/data/derivatives/registration/epi2t1weighted/struct2mnimat_ants/_subject_id_{subject_id}/transformComposite.h5',
#              'mask':fsl.Info.standard_image('MNI152_T1_1mm_brain_mask.nii.gz')}

In [12]:
# # Copes at level 2:
# #

# # In case you have only one cope at level 2, specify it on the input anyway.
# #Cope number from level 1
# lvl1_cope=int(sys.argv[1])
# #Cope number from level 2
# lvl2_cope=int(sys.argv[2])


# # The directory where we can find the level 2 output
# l12_out_dir=os.path.join(project_dir,"level2s","model"+model_id)

# wf = pe.Workflow(name='wf')
# wf.base_dir = os.path.join(work_dir,"wdir"+str(model_id)+"lvl3","copes_"+str(lvl1_cope)+"_"+str(lvl2_cope))
# wf.config = {"execution": {"crashdump_dir":os.path.join(project_dir,'crashdumps')}}